<a href="https://colab.research.google.com/github/calebmk21/CS-260D/blob/main/DataSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Set-up:

In [ ]:
!pip install opacus
!pip install apricot-select numpy scikit-learn
!pip install kmedoids

In [ ]:
# For the CNN + DP-SGD
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Subset
import numpy as np
from opacus import PrivacyEngine
import time
import matplotlib.pyplot as plt

#For Lazy Greedy
from sklearn.metrics import pairwise_distances
import heapq
from sklearn.metrics import pairwise_distances
from apricot import FacilityLocationSelection
import time

#For K-medoids
import kmedoids
from sklearn.metrics.pairwise import euclidean_distances

# Model

In [ ]:
# baseline model
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        #=======================================#
        #            MNIST Settings
        #=======================================#

        # self.conv1 = nn.Conv2d(1, 16, 3, 1)
        # self.conv2 = nn.Conv2d(16, 32, 3, 1)
        # self.fc1 = nn.Linear(32*12*12, 64)
        # self.fc2 = nn.Linear(64, 10)

        #=======================================#
        #            CIFAR-10 Settings
        #=======================================#

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=576, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):
        # x = F.relu(self.conv1(x))
        # x = F.relu(self.conv2(x))
        # x = F.max_pool2d(x, 2)
        # x = torch.flatten(x, 1)
        # x = F.relu(self.fc1(x))
        # return self.fc2(x)

        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
# MNIST Model
class CNN_MNIST(nn.Module):
    def __init__(self):
        super().__init__()

        #=======================================#
        #            MNIST Settings
        #=======================================#

        self.conv1 = nn.Conv2d(1, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32*12*12, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


# CIFAR-10 Model

class CNN_CIFAR10(nn.Module):
    def __init__(self):
        super().__init__()

        #=======================================#
        #            CIFAR-10 Settings
        #=======================================#

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=576, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):

        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
def train_one_epoch(model, loader, optimizer):
    # train loop
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.cross_entropy(out, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [ ]:
@torch.no_grad()
def evaluate(model, loader):
    # evaluation
    model.eval()
    loss, correct = 0, 0
    for data, target in loader:
        data, target = data.to(device), target.to(device)
        out = model(data)
        loss += F.cross_entropy(out, target, reduction="sum").item()
        pred = out.argmax(1)
        correct += pred.eq(target).sum().item()
    loss /= len(loader.dataset)
    acc = 100. * correct / len(loader.dataset)
    return loss, acc

In [ ]:
def run_experiment(dataset='MNIST', use_dp=False, fixed_privacy_budget=False):

    # setup model and optimizer
    if dataset == 'MNIST':
      model = CNN_MNIST().to(device)
    elif dataset == "CIFAR":
      model = CNN_CIFAR10().to(device)

    # Defaults to base CNN class if no dataset is specified
    else:
      model = CNN().to(device)

    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)

    privacy_engine = None
    if use_dp:

        if fixed_privacy_budget:

            privacy_engine = PrivacyEngine()

            # Calculates sigma based on target epsilon and delta
            model, optimizer, train_loader_dp = privacy_engine.make_private_with_epsilon(
              module=model,
              optimizer=optimizer,
              data_loader=train_loader,
              max_grad_norm=MAX_GRAD_NORM,
              target_delta=TARGET_DELTA,
              target_epsilon=TARGET_EPSILON,
              epochs=NUM_EPOCHS
        )
        else:

            privacy_engine = PrivacyEngine()

            # Otherwise takes sigma as a hyperparameter
            model, optimizer, train_loader_dp = privacy_engine.make_private(
              module=model,
              optimizer=optimizer,
              data_loader=train_loader,
              max_grad_norm=MAX_GRAD_NORM,
              noise_multiplier=NOISE_MULTIPLIER,
        )
    else:
        train_loader_dp = train_loader


    # train
    best_val_acc = 0
    epochs_no_improve = 0
    best_model_path = f"best_model{'_dp' if use_dp else ''}.pt"

    start_time = time.time()

    for epoch in range(1, EPOCHS + 1):
        train_loss = train_one_epoch(model, train_loader_dp, optimizer)
        val_loss, val_acc = evaluate(model, val_loader)

        eps = privacy_engine.get_epsilon(DELTA)

        print(f"[{'DP-SGD' if use_dp else 'Standard SGD'}] Epoch {epoch}: "
            + f"train_loss={train_loss:.4f}, "
            + f"val_loss={val_loss:.4f}, val_acc={val_acc:.4f}%"
            + f", ε={eps:.4f}" if use_dp else "")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            epochs_no_improve = 0
            torch.save(model.state_dict(), best_model_path)
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= PATIENCE:
            print(f"\nEarly stopping at epoch {epoch} (no improvement for {PATIENCE} epochs).")
            break


    end_time = time.time()
    total_time = end_time - start_time

    # test on best model
    model.load_state_dict(torch.load(best_model_path))
    test_loss, test_acc = evaluate(model, test_loader)
    final_eps = privacy_engine.get_epsilon(DELTA) if use_dp else None

    print(f"\n[{ 'DP-SGD' if use_dp else 'Standard-SGD'}]")
    print(f"Best val acc: {best_val_acc:.4f}%, Test acc: {test_acc:.4f}%\n" + (f" ε={final_eps:.4f}\n" if use_dp else "") + (f"Total runtime: {total_time:.4f} seconds"))
    return best_val_acc, test_acc, final_eps

# Model Hyperparameters

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 128
LR = 0.01
EPOCHS = 20
SEED = 42
MAX_GRAD_NORM = 1.0
DELTA = 1e-5
PATIENCE = 5

# For dynamically calculated epsilon
NOISE_MULTIPLIER = 1.0

# For a fixed privacy budget (eps, del)
TARGET_EPSILON = 8
TARGET_DELTA = 1e-5
NUM_EPOCHS = 20

torch.manual_seed(SEED)
np.random.seed(SEED)

# Data Selection

In [ ]:
def select_dataset_greedy(name, n_samples):
  if name == "MNIST":
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),])

    full_train = datasets.MNIST(
        root="./data",
        train=True,
        download=True,
        transform=transform
    )

    train_size = 5000
    val_size = 1000
    train_dataset, subset_rest = random_split(full_train, [train_size, len(full_train) - train_size])
    val_dataset, _ = random_split(subset_rest, [val_size, len(subset_rest) - val_size])

    test_dataset = datasets.MNIST(
        root="./data",
        train=False,
        download=True,
        transform=transform
    )

  elif name == "CIFAR":
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])


    full_train = datasets.CIFAR10(
        root="./data",
        train=True,
        download=True,
        transform=transform
    )

    train_size = 40000
    val_size = len(full_train) - train_size
    train_dataset, subset_rest = random_split(full_train, [train_size, len(full_train) - train_size])
    val_dataset, _ = random_split(subset_rest, [val_size, len(subset_rest) - val_size])


    test_dataset = datasets.CIFAR10(
        root="./data",
        train=False,
        download=True,
        transform=transform
    )

  train_data_array = train_dataset.dataset.data[train_dataset.indices]
  # Convert to numpy array first if it's a tensor, then convert type
  if isinstance(train_data_array, torch.Tensor):
    train_data_array = train_data_array.cpu().numpy()

  selector = FacilityLocationSelection(n_samples=n_samples, metric='euclidean', optimizer='lazy', verbose=True)

  # Reshape the subset for selection
  train_data_array = train_data_array.reshape(train_data_array.shape[0], -1)

  # Use the subset for selection
  selector.fit(train_data_array)

  selected_indices = selector.ranking

  selected_train_dataset = Subset(train_dataset, selected_indices)

  train_loader = DataLoader(selected_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
  val_loader   = DataLoader(val_dataset, batch_size=256, shuffle=False)
  test_loader  = DataLoader(test_dataset, batch_size=256, shuffle=False)

  return train_loader, val_loader, test_loader

In [ ]:
def select_dataset_kmedoids(name, n_clusters):
  if name == "MNIST":
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),])

    full_train = datasets.MNIST(
        root="./data",
        train=True,
        download=True,
        transform=transform
    )

    train_size = 50000
    val_size = len(full_train) - train_size
    train_dataset, val_dataset = random_split(full_train, [train_size, val_size])

    test_dataset = datasets.MNIST(
        root="./data",
        train=False,
        download=True,
        transform=transform
    )

  elif name == "CIFAR":
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])


    full_train = datasets.CIFAR10(
        root="./data",
        train=True,
        download=True,
        transform=transform
    )

    train_size = 40000
    val_size = len(full_train) - train_size
    train_dataset, val_dataset = random_split(full_train, [train_size, val_size])


    test_dataset = datasets.CIFAR10(
        root="./data",
        train=False,
        download=True,
        transform=transform
    )


  train_data_array = train_dataset.dataset.data[train_dataset.indices]
  # Convert to numpy array first if it's a tensor, then convert type
  if isinstance(train_data_array, torch.Tensor):
    train_data_array = train_data_array.cpu().numpy()
  train_data_array = train_data_array.astype('float32')
  train_data_flat = train_data_array.reshape(train_data_array.shape[0], -1)

  # Create a smaller subset for kmedoids to prevent crashes due to large memory usage
  subset_size_for_kmedoids = min(5000, train_data_array.shape[0]) # Adjusted subset size
  np.random.seed(SEED) # Ensure reproducibility
  random_indices_kmedoids = np.random.choice(train_data_array.shape[0], subset_size_for_kmedoids, replace=False)
  subset_for_kmedoids = train_data_flat[random_indices_kmedoids]

  # Calculate distances only for the subset
  diss = euclidean_distances(subset_for_kmedoids)

  kmeds = kmedoids.KMedoids(n_clusters=n_clusters, method='fasterpam', random_state=0, metric='precomputed')
  model = kmeds.fit(diss)

  # Map the medoid indices from the subset back to the original dataset indices
  selected_subset_indices = model.medoid_indices_
  selected_original_indices = random_indices_kmedoids[selected_subset_indices]
  select = [train_dataset.indices[i] for i in selected_original_indices]

  selected_train_dataset = Subset(full_train, select)

  train_loader = DataLoader(selected_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
  val_loader   = DataLoader(val_dataset, batch_size=256, shuffle=False)
  test_loader  = DataLoader(test_dataset, batch_size=256, shuffle=False)

  return train_loader, val_loader, test_loader

# Test:

## MNIST

### Lazy Greedy

In [ ]:
DATASET_NAME = "MNIST"
N_SAMPLES = 100
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for Lazy Greedy (100 samples)")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

In [ ]:
DATASET_NAME = "MNIST"
N_SAMPLES = 200
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for  Lazy Greedy (200 samples)")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

In [ ]:
DATASET_NAME = "MNIST"
N_SAMPLES = 300
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for Lazy Greedy (300 samples)")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

### K-Medoids

In [ ]:
DATASET_NAME = "MNIST"
N_CLUSTERS = 100
train_loader, val_loader, test_loader = select_dataset_kmedoids(DATASET_NAME, N_CLUSTERS)

print(f"\nRunning DP-SGD baseline for K-Medoids (100 cluster)")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

In [ ]:
DATASET_NAME = "MNIST"
N_CLUSTERS = 200
train_loader, val_loader, test_loader = select_dataset_kmedoids(DATASET_NAME, N_CLUSTERS)

print(f"\nRunning DP-SGD baseline for K-Medoids (200 cluster)")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

In [ ]:
DATASET_NAME = "MNIST"
N_CLUSTERS = 300
train_loader, val_loader, test_loader = select_dataset_kmedoids(DATASET_NAME, N_CLUSTERS)

print(f"\nRunning DP-SGD baseline for K-Medoids (300 cluster)")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

## CIFAR-10

### Lazy Greedy

In [ ]:
DATASET_NAME = "CIFAR"

N_SAMPLES = 50
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for Lazy Greedy (50 samples)")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

In [ ]:
DATASET_NAME = "CIFAR"
N_SAMPLES = 100
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for Lazy Greedy (100 samples)")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

In [ ]:
DATASET_NAME = "CIFAR"
N_SAMPLES = 200
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for Lazy Greedy (200 samples)")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

In [ ]:
DATASET_NAME = "CIFAR"
N_SAMPLES = 300
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for Lazy Greedy (300 samples)")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

In [ ]:
DATASET_NAME = "CIFAR"
N_SAMPLES = 500
train_loader, val_loader, test_loader = select_dataset_greedy(DATASET_NAME, N_SAMPLES)

print(f"\nRunning DP-SGD baseline for Lazy Greedy (500 samples)")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

### K-Medoids

In [ ]:
DATASET_NAME = "CIFAR"
N_CLUSTERS = 100
train_loader, val_loader, test_loader = select_dataset_kmedoids(DATASET_NAME, N_CLUSTERS)

print(f"\nRunning DP-SGD baseline for K-Medoids (100 cluster)")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

In [ ]:
DATASET_NAME = "CIFAR"
N_CLUSTERS = 200
train_loader, val_loader, test_loader = select_dataset_kmedoids(DATASET_NAME, N_CLUSTERS)

print(f"\nRunning DP-SGD baseline for K-Medoids (200 cluster)")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

In [ ]:
DATASET_NAME = "CIFAR"
N_CLUSTERS = 300
train_loader, val_loader, test_loader = select_dataset_kmedoids(DATASET_NAME, N_CLUSTERS)

print(f"\nRunning DP-SGD baseline for K-Medoids (300 cluster)")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)

In [ ]:
DATASET_NAME = "CIFAR"
N_CLUSTERS = 500
train_loader, val_loader, test_loader = select_dataset_kmedoids(DATASET_NAME, N_CLUSTERS)

print(f"\nRunning DP-SGD baseline for K-Medoids (500 cluster)")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)